# 07 经典 CNN 架构

> LeNet → AlexNet → VGG → ResNet 的演进历史

---

## 📚 学习目标

- [ ] 了解 CNN 发展历史
- [ ] 理解各架构的核心创新
- [ ] 用 PyTorch 复现经典架构

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

try:
    import torch
    import torch.nn as nn
    HAS_TORCH = True
except ImportError:
    HAS_TORCH = False
    print("请安装 PyTorch")

---

## 📜 CNN 发展时间线

```
1998: LeNet-5 (Yann LeCun)
      ↓ 手写识别成功
      
2012: AlexNet (Alex Krizhevsky)
      ↓ ImageNet 冠军，GPU 训练
      
2014: VGGNet (Oxford)
      ↓ 深度的力量，3×3 卷积
      
2014: GoogLeNet/Inception (Google)
      ↓ 多尺度特征
      
2015: ResNet (Microsoft)
      ↓ 残差连接，152 层！
      
2019: EfficientNet (Google)
      ↓ 复合缩放
```

---

## 🏛️ 1. LeNet-5 (1998)

**论文**: Gradient-Based Learning Applied to Document Recognition

**架构**:
```
输入 (1, 32, 32)
  ↓
Conv (6, 5×5) → Sigmoid → AvgPool (2×2)
  ↓ (6, 14, 14)
Conv (16, 5×5) → Sigmoid → AvgPool (2×2)
  ↓ (16, 5, 5)
FC (400 → 120) → Sigmoid
FC (120 → 84) → Sigmoid
FC (84 → 10)
```

**特点**:
- 第一个成功的 CNN
- 用于银行支票数字识别
- 使用 Sigmoid（现在用 ReLU）

In [ ]:
if HAS_TORCH:
    class LeNet5(nn.Module):
        """LeNet-5 的 PyTorch 实现"""
        
        def __init__(self):
            super().__init__()
            self.conv1 = nn.Conv2d(1, 6, 5)
            self.conv2 = nn.Conv2d(6, 16, 5)
            self.fc1 = nn.Linear(16 * 5 * 5, 120)
            self.fc2 = nn.Linear(120, 84)
            self.fc3 = nn.Linear(84, 10)
        
        def forward(self, x):
            x = torch.sigmoid(self.conv1(x))
            x = nn.functional.avg_pool2d(x, 2)
            x = torch.sigmoid(self.conv2(x))
            x = nn.functional.avg_pool2d(x, 2)
            x = x.view(-1, 16 * 5 * 5)
            x = torch.sigmoid(self.fc1(x))
            x = torch.sigmoid(self.fc2(x))
            x = self.fc3(x)
            return x
    
    lenet = LeNet5()
    params = sum(p.numel() for p in lenet.parameters())
    print(f"LeNet-5 参数量: {params:,}")

---

## 🏛️ 2. AlexNet (2012)

**论文**: ImageNet Classification with Deep Convolutional Neural Networks

**核心创新**:
- ReLU 激活函数
- Dropout 正则化
- 数据增强
- GPU 训练
- Local Response Normalization (现已弃用)

**影响**: 开启深度学习时代！

---

## 🏛️ 3. VGGNet (2014)

**论文**: Very Deep Convolutional Networks for Large-Scale Image Recognition

**核心创新**:
- 全部使用 3×3 卷积核
- 两个 3×3 = 一个 5×5 的感受野
- 更深（16-19 层）

**架构模式**:
```
[Conv 3×3, 64] × 2 → MaxPool
[Conv 3×3, 128] × 2 → MaxPool
[Conv 3×3, 256] × 3 → MaxPool
[Conv 3×3, 512] × 3 → MaxPool
[Conv 3×3, 512] × 3 → MaxPool
FC → FC → FC
```

---

## 🏛️ 4. ResNet (2015) ⭐最重要

**论文**: Deep Residual Learning for Image Recognition

**核心创新**: 残差连接

```
普通网络:  x → [层] → F(x)

残差网络:  x → [层] → F(x) + x
           ↓              ↑
           └──────────────┘ (跳跃连接)
```

**为什么有效**:
1. 梯度可以通过跳跃连接直接回传
2. 学习 F(x) = 0 很容易（输出 = 输入）
3. 可以训练非常深的网络（152 层）

In [ ]:
if HAS_TORCH:
    class ResidualBlock(nn.Module):
        """残差块"""
        
        def __init__(self, channels):
            super().__init__()
            self.conv1 = nn.Conv2d(channels, channels, 3, padding=1)
            self.bn1 = nn.BatchNorm2d(channels)
            self.conv2 = nn.Conv2d(channels, channels, 3, padding=1)
            self.bn2 = nn.BatchNorm2d(channels)
        
        def forward(self, x):
            identity = x  # 保存输入
            
            out = self.conv1(x)
            out = self.bn1(out)
            out = nn.functional.relu(out)
            
            out = self.conv2(out)
            out = self.bn2(out)
            
            out = out + identity  # 残差连接！
            out = nn.functional.relu(out)
            
            return out
    
    # 测试
    block = ResidualBlock(64)
    x = torch.randn(1, 64, 32, 32)
    y = block(x)
    print(f"输入形状: {x.shape}")
    print(f"输出形状: {y.shape}")
    print("\n💡 输入输出形状相同，这就是残差连接的特点！")

---

## 📊 架构对比

| 架构 | 年份 | 层数 | Top-5 错误率 | 参数量 | 核心创新 |
|------|------|------|-------------|--------|----------|
| LeNet | 1998 | 7 | - | 60K | 第一个 CNN |
| AlexNet | 2012 | 8 | 16.4% | 60M | ReLU, Dropout |
| VGG-16 | 2014 | 16 | 7.3% | 138M | 3×3 卷积 |
| GoogLeNet | 2014 | 22 | 6.7% | 5M | Inception 模块 |
| ResNet-152 | 2015 | 152 | 3.6% | 60M | 残差连接 |

---

## 📝 本章小结

### 关键进步

1. **激活函数**: Sigmoid → ReLU
2. **正则化**: 无 → Dropout → BatchNorm
3. **深度**: 7层 → 152层（残差连接）
4. **卷积核**: 大核 → 小核堆叠

### 实用建议

- 从 ResNet 开始，它是现代 CNN 的基础
- 使用预训练模型（下一章）

---

## 🚀 下一步

继续学习 **08_transfer_learning.ipynb** - 迁移学习